In [ ]:
!wget https://setup.johnsnowlabs.com/colab.sh -O - | bash /dev/stdin -p 3.4.1 -s 5.1.2 -g

--2023-12-10 16:59:09--  https://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-12-10 16:59:09--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-12-10 16:59:09 (119 MB/s) - written to stdout [1191/1191]

Installin

In [ ]:
import pandas as pd

In [ ]:
file_path = 'annotated-titles.csv'
df = pd.read_csv(file_path,encoding='latin1')

In [ ]:
df.head(3)

,title,label,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,To require the Secretary of Agriculture to can...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,To require the prompt reconstruction of and re...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Expressing support for the designation of Sept...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
unique_labels = df['label'].unique()
print(unique_labels)
label_counts = df['label'].value_counts()
print(label_counts)

['-' '1' '0' nan '#VALUE!' '-1' ' and for other purposes."']
-                            232487
0                             23262
1                               667
-1                               76
 and for other purposes."         2
#VALUE!                           1
Name: label, dtype: int64


In [ ]:
allowed_values = ['-', '0', '1']
df = df[df['label'].isin(allowed_values)]

In [ ]:
selected_columns = ['title', 'label']
df = df[selected_columns]

In [ ]:
df['label'] = df['label'].replace('-', '0')
label_counts = df['label'].value_counts()
print(label_counts)
df.head()

0    255749
1       667
Name: label, dtype: int64


,title,label
0,To require the Secretary of Agriculture to can...,0
1,To require the prompt reconstruction of and re...,0
2,Expressing support for the designation of Sept...,0
3,To amend the State Department Basic Authoritie...,0
4,Electing Members to the Joint Committee of Con...,0


In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
import os
import sys

import sparknlp

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import pandas as pd

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.1.2
Apache Spark version:  3.2.3


In [ ]:
dataDF = spark.read \
      .option("header", True) \
      .csv("data.csv")

dataDF.show(truncate=50)

+--------------------------------------------------+-----+
|                                             title|label|
+--------------------------------------------------+-----+
|To require the Secretary of Agriculture to canc...|    0|
|To require the prompt reconstruction of and res...|    0|
|Expressing support for the designation of Septe...|    0|
|To amend the State Department Basic Authorities...|    0|
|Electing Members to the Joint Committee of Cong...|    0|
|To authorize the Bureau of Reclamation to parti...|    0|
|To reauthorize and amend title XIV of the Publi...|    0|
|To amend the National Wool Act of 1954 to reduc...|    0|
|To amend the Act establishing Golden Gate Natio...|    0|
|An act to amend the National Wool Act of 1954 t...|    0|
|An act to amend the National Wool Act of 1954 t...|    0|
|To provide for the use of Department of Defense...|    0|
|To make systematic and comprehensive reductions...|    0|
|To amend the National Labor Relations Act and t...|    

In [ ]:
from pyspark.sql.functions import col

dataDF.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+------+
|               label| count|
+--------------------+------+
|                   0|255725|
|                   1|   667|
| 166 degrees 34'1...|     5|
|       1789-1993""."|     4|
|      166 degrees 34|     1|
| and for other pu...|     1|
|             (4""r)-|     1|
|              Design|     1|
|       1853-1861""."|     1|
|       1917-1990""."|     1|
| The Committee on...|     1|
| sponsored by the...|     1|
| launched by Pres...|     1|
|       1870-1989""."|     1|
|5-pentanediyl) bi...|     1|
| as a party to th...|     1|
| and for other pu...|     1|
| the publication ...|     1|
|2-ethane-diyl-bis...|     1|
+--------------------+------+



In [ ]:
dataDF = dataDF.withColumn("label", dataDF["label"].cast("int"))

filtered_dataDF = dataDF.filter((col("label") == "0") | (col("label") == "1"))

filtered_dataDF.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+-----+------+
|label| count|
+-----+------+
|    0|255725|
|    1|   667|
+-----+------+



Using Logistic regression

In [ ]:
from pyspark.ml.feature import StringIndexer
document_assembler = DocumentAssembler() \
      .setInputCol("title") \
      .setOutputCol("document")

loaded_useEmbeddings = UniversalSentenceEncoder.load('/root/cache_pretrained/tfhub_use_en_2.4.0_2.4_1587136330099')\
      .setInputCols("document")\
      .setOutputCol("use_embeddings")

embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["use_embeddings"]) \
      .setOutputCols(["finished_use_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

use_pipeline = Pipeline(
      stages=[
        document_assembler,
        loaded_useEmbeddings,
        embeddings_finisher,
        ]
      )

use_df = use_pipeline.fit(filtered_dataDF).transform(filtered_dataDF)

In [ ]:
use_df.show(2)

+--------------------+-----+--------------------+--------------------+-----------------------+
|               title|label|            document|      use_embeddings|finished_use_embeddings|
+--------------------+-----+--------------------+--------------------+-----------------------+
|To require the Se...|    0|[{document, 0, 75...|[{sentence_embedd...|   [[-0.007805520202...|
|To require the pr...|    0|[{document, 0, 12...|[{sentence_embedd...|   [[-0.022778689861...|
+--------------------+-----+--------------------+--------------------+-----------------------+
only showing top 2 rows



In [ ]:
from pyspark.sql.functions import explode
use_df= use_df.withColumn("features", explode(use_df.finished_use_embeddings))
use_df.show(2)

+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
|               title|label|            document|      use_embeddings|finished_use_embeddings|            features|
+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
|To require the Se...|    0|[{document, 0, 75...|[{sentence_embedd...|   [[-0.007805520202...|[-0.0078055202029...|
|To require the pr...|    0|[{document, 0, 12...|[{sentence_embedd...|   [[-0.022778689861...|[-0.0227786898612...|
+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
only showing top 2 rows



In [ ]:
label_distribution = use_df.groupBy("label").count().collect()

train_percentage = 0.7
test_percentage = 0.3

trainingData = spark.createDataFrame([], schema=use_df.schema)
testData = spark.createDataFrame([], schema=use_df.schema)

for row in label_distribution:
    label = row['label']
    count = row['count']

    train_count = int(count * train_percentage)
    test_count = count - train_count

    label_data = use_df.filter(use_df['label'] == label)
    label_train_data, label_test_data = label_data.randomSplit([train_percentage, test_percentage], seed=100)

    trainingData = trainingData.union(label_train_data)
    testData = testData.union(label_test_data)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 179109
Test Dataset Count: 77283


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

In [ ]:
df = predictions.select('title','label','prediction').toPandas()
df.head()

,title,label,prediction
0,An act to make technical corrections to title ...,1,0.0
1,"To make technical corrections to title 23, Uni...",1,0.0
2,An Act Making appropriations for the Departmen...,1,0.0
3,An Act To amend the Clean Air Act to remove th...,1,0.0
4,An Act To amend the Family Educational Rights ...,1,0.0


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(df.label, df.prediction))
print(accuracy_score(df.label, df.prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     77071
           1       0.00      0.00      0.00       212

    accuracy                           1.00     77283
   macro avg       0.50      0.50      0.50     77283
weighted avg       0.99      1.00      1.00     77283

0.9972568352677821


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Using ClassifierDL

In [ ]:
document = DocumentAssembler()\
    .setInputCol("title")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.load('/root/cache_pretrained/tfhub_use_en_2.4.0_2.4_1587136330099')\
      .setInputCols("document")\
      .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("label")\
    .setMaxEpochs(5)\
    .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

In [ ]:
pipelineModel = pipeline.fit(trainingData)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

df = pipelineModel.transform(testData).select('title','label',"class.result").toPandas()

df['result'] = df['result'].apply(lambda x: x[0])

In [ ]:
y_true = df.label
y_pred_int = [int(label) for label in df.result]
print(classification_report(y_true, y_pred_int))
print(accuracy_score(y_true, y_pred_int))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     77071
           1       0.00      0.00      0.00       212

    accuracy                           1.00     77283
   macro avg       0.50      0.50      0.50     77283
weighted avg       0.99      1.00      1.00     77283

0.9972568352677821


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Downsampling label 0 and using ClassifierDL**


In [ ]:
from pyspark.sql.functions import col

num_label_1 = 667
num_label_0 = 1800

label_1_df = filtered_dataDF.filter(col("label") == 1)

sampled_label_1_df = label_1_df.sample(False, num_label_1 / label_1_df.count())

label_0_df = filtered_dataDF.filter(col("label") == 0)

sampled_label_0_df = label_0_df.sample(False, num_label_0 / label_0_df.count())

final_sampled_df = sampled_label_0_df.union(sampled_label_1_df)

final_sampled_df.groupBy("label") \
    .count() \
    .show()


+-----+-----+
|label|count|
+-----+-----+
|    0| 1775|
|    1|  667|
+-----+-----+



In [ ]:
selected_dataDF = final_sampled_df.select("title", "label")

In [ ]:
selected_dataDF

DataFrame[title: string, label: int]

In [ ]:
document = DocumentAssembler()\
    .setInputCol("title")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.load('/root/cache_pretrained/tfhub_use_en_2.4.0_2.4_1587136330099')\
      .setInputCols("document")\
      .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("label")\
    .setMaxEpochs(5)\
    .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

In [ ]:
trainingData

DataFrame[title: string, label: int]

In [ ]:
label_distribution = selected_dataDF.groupBy("label").count().collect()

train_percentage = 0.7
test_percentage = 0.3

trainingData = spark.createDataFrame([], schema=selected_dataDF.schema)
testData = spark.createDataFrame([], schema=selected_dataDF.schema)

for row in label_distribution:
    label = row['label']
    count = row['count']

    train_count = int(count * train_percentage)
    test_count = count - train_count

    label_data = selected_dataDF.filter(selected_dataDF['label'] == label)
    label_train_data, label_test_data = label_data.randomSplit([train_percentage, test_percentage], seed=100)

    trainingData = trainingData.union(label_train_data)
    testData = testData.union(label_test_data)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 1663
Test Dataset Count: 779


In [ ]:
pipelineModel = pipeline.fit(trainingData)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

df = pipelineModel.transform(testData).select('title','label',"class.result").toPandas()

df['result'] = df['result'].apply(lambda x: x[0])

In [ ]:
y_true = df.label
y_pred_int = [int(label) for label in df.result]
print(classification_report(y_true, y_pred_int))
print(accuracy_score(y_true, y_pred_int))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94       585
           1       0.82      0.86      0.84       194

    accuracy                           0.92       779
   macro avg       0.89      0.90      0.89       779
weighted avg       0.92      0.92      0.92       779

0.9178433889602053
